# Business Case -- XPTO LLC.

## Data Analyst

### Candidate: Rafael Feltrin

Let's start by importing functions from the modules I wrote to perform the challenge's Parts I and II, which are respectively *etl_script.py* and *eda_script.py*.

In [1]:
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)


from scripts.eda_script import (slice_date)
from scripts.etl_script import (create_geodf,
                                read_csv, 
                                write_associates_and_branches_cols,
                                write_database_dates)


# Part I : ETL

We must start by opening the .csv file and creating some new columns as we see fit -- they might be used here or could be nice to have in future analyses.

So we begin we the *read_csv* method, which takes the file path and already does a simple change, which is filling the root CNPJ and the full CNPJ with leading zeroes in case they are missing.

I always do that because this kind of data tends to end up in the hands of less tech-savvy areas such as sales ops and marketing, and it is very common for them to match these CNPJ columns with manually collected spreadsheets or CRM outputs which commonly has the leading zeroes.

In [2]:
path = r'C:\Users\rafaf\PycharmProjects\parana_irs_dataset_analysis\data\data_case_2024_03.csv'

df = read_csv(path=path,
              sep=',',
              zfill_cols=['document_number', 'cnpj_basico'])

display(df.head(10))

,document_number,cnpj_basico,establishment_type,razao_social,nome_fantasia,opening_date,cnae,cnae_description,mcc,mcc_description,total_branches_and_associates,city_state,city_code,zip_code,share_capital,size_company,juri_description,juri_description_ENG,email_provider
0,79690152000466,79690152,FILIAL,COMERCIAL AGRICOLA DE PARANAVAI LTDA,KATO TRATORES E IMPLEMENTOS,26 de novembro de 2003,4661300,"Comércio atacadista de máquinas, aparelhos e e...",5599,"Miscellaneous Automotive, Aircraft, and Farm E...","{""total_associates"": 8, ""total_branches"": ""12 ...",Ubiratã - PR,4128005,85440000,4750000.0,OTHER,Sociedade Empresária Limitada,Limited Liability Company (LLC),NaN
1,79690152000385,79690152,FILIAL,COMERCIAL AGRICOLA DE PARANAVAI LTDA,COMERCIAL AGRICOLA DE PARANAVAI LTDA,17 de setembro de 1976,4661300,"Comércio atacadista de máquinas, aparelhos e e...",5599,"Miscellaneous Automotive, Aircraft, and Farm E...","{""total_associates"": 8, ""total_branches"": ""12 ...",Loanda - PR,4113502,87900000,4750000.0,OTHER,Sociedade Empresária Limitada,Limited Liability Company (LLC),NaN
2,79690152000628,79690152,FILIAL,COMERCIAL AGRICOLA DE PARANAVAI LTDA,KATO TRATORES E IMPLEMENTOS,10 de janeiro de 1995,4661300,"Comércio atacadista de máquinas, aparelhos e e...",5599,"Miscellaneous Automotive, Aircraft, and Farm E...","{""total_associates"": 8, ""total_branches"": ""12 ...",Maringá - PR,4115200,87065005,4750000.0,OTHER,Sociedade Empresária Limitada,Limited Liability Company (LLC),katoagricola.com.br
3,79690152000709,79690152,FILIAL,COMERCIAL AGRICOLA DE PARANAVAI LTDA,KATO TRATORES E IMPLEMENTOS,22 de junho de 1999,4661300,"Comércio atacadista de máquinas, aparelhos e e...",5599,"Miscellaneous Automotive, Aircraft, and Farm E...","{""total_associates"": 8, ""total_branches"": ""12 ...",Umuarama - PR,4128104,87507011,4750000.0,OTHER,Sociedade Empresária Limitada,Limited Liability Company (LLC),katoagricola.com.br
4,79690152000113,79690152,MATRIZ,COMERCIAL AGRICOLA DE PARANAVAI LTDA,NaN,22 de novembro de 1968,4661300,"Comércio atacadista de máquinas, aparelhos e e...",5599,"Miscellaneous Automotive, Aircraft, and Farm E...","{""total_associates"": 8, ""total_branches"": ""12 ...",Paranavaí - PR,4118402,87711000,4750000.0,OTHER,Sociedade Empresária Limitada,Limited Liability Company (LLC),KATOAGRICOLA.COM.BR
5,79690152000202,79690152,FILIAL,COMERCIAL AGRICOLA DE PARANAVAI LTDA,KATO TRATORES E IMPLEMENTOS,09 de outubro de 2003,4661300,"Comércio atacadista de máquinas, aparelhos e e...",5599,"Miscellaneous Automotive, Aircraft, and Farm E...","{""total_associates"": 8, ""total_branches"": ""12 ...",Campo Mourão - PR,4104303,87308550,4750000.0,OTHER,Sociedade Empresária Limitada,Limited Liability Company (LLC),katoagricola.com.br
6,79690152000890,79690152,FILIAL,COMERCIAL AGRICOLA DE PARANAVAI LTDA,KATO TRATORES,01 de novembro de 2011,4661300,"Comércio atacadista de máquinas, aparelhos e e...",5599,"Miscellaneous Automotive, Aircraft, and Farm E...","{""total_associates"": 8, ""total_branches"": ""12 ...",Goioerê - PR,4108601,87360000,4750000.0,OTHER,Sociedade Empresária Limitada,Limited Liability Company (LLC),KATOAGRICOLA.COM.BR
7,01092817001333,01092817,FILIAL,M. A. MAQUINAS AGRICOLAS LTDA.,M.A. MAQUINAS,19 de dezembro de 2016,4661300,"Comércio atacadista de máquinas, aparelhos e e...",5599,"Miscellaneous Automotive, Aircraft, and Farm E...","{""total_associates"": 8, ""total_branches"": ""13 ...",Cascavel - PR,4104808,85804600,168546832.0,OTHER,Sociedade Empresária Limitada,Limited Liability Company (LLC),NaN
8,01092817000795,01092817,FILIAL,M. A. MAQUINAS AGRICOLAS LTDA.,M. A. MAQUINAS,27 de outubro de 2008,4661300,"Comércio atacadista de máquinas, aparelhos e e...",5599,"Miscellaneous Automotive, Aircraft, and Farm E...","{""total_associates"": 8, ""total_branches"": ""13 ...",Ubiratã - PR,4128005,85440000,168546832.0,OTHER,Sociedade Empresária Limitada,Limited Liability Company (LLC),NaN
9,01092817001252,01092817,FILIAL,M. A. MAQUINAS A

We can see it works.

Next, we will convert the string-formatted to a database-friendly date format just in case we need it later. The *write_database_dates* function does that in a very modular fashion -- if we went back to the Roman calendar or added a new month we could only update the inputs!

Back to the real world, we input the 12 months in Portuguese and run it.

In [3]:
portuguese_months = ['janeiro', 'fevereiro', 'março', 'abril',
                     'maio', 'junho', 'julho', 'agosto', 
                     'setembro', 'outubro', 'novembro', 'dezembro']

month_numbers = [f"{i:02d}" for i in range(1, len(portuguese_months) + 1)]

months = {name: num for name, num in zip(portuguese_months, month_numbers)}

df = write_database_dates(df=df, 
                          date_col='opening_date', 
                          months=months)

display(df['formatted_opening_date'].head(10))

0    20031126
1    19760917
2    19950110
3    19990622
4    19681122
5    20031009
6    20111101
7    20161219
8    20081027
9    20110803
Name: formatted_opening_date, dtype: object

Another interesting column is the one with the amount of branches and associated. It is a JSON-like structure, and we parse it with the *write_associates_and_branches_cols* to add two new integer columns with the number of associates and branches.  

In [4]:
df = write_associates_and_branches_cols(df=df,
                                        json_col='total_branches_and_associates')

display(df[['total_associates', 'total_branches']].head(10))

,total_associates,total_branches
0,8,12
1,8,12
2,8,12
3,8,12
4,8,12
5,8,12
6,8,12
7,8,13
8,8,13
9,8,13


Finally, the cherry on top. Using the *geobr* package we can get official geospatial data from IPEA, which we will need when plotting map charts in Part II.

The *create_geodf* function reads all IBGE city codes in the dataset and gets their geometry in a GeoDataFrame format, which we will later join with a pivot table of company data to plot the maps and show business insights.   

In [5]:
gdf = create_geodf(df=df,
                   city_code_col='city_code',
                   year=2022)

display(gdf.head(10))

,code_muni,name_muni,code_state,abbrev_state,name_state,code_region,name_region,geometry
0,4128005.0,Ubiratã,41.0,PR,Paraná,4.0,Sul,"MULTIPOLYGON (((-52.97501 -24.34684, -52.98215..."
1,4113502.0,Loanda,41.0,PR,Paraná,4.0,Sul,"MULTIPOLYGON (((-53.04921 -23.09932, -53.04181..."
2,4115200.0,Maringá,41.0,PR,Paraná,4.0,Sul,"MULTIPOLYGON (((-51.89955 -23.30343, -51.89638..."
3,4128104.0,Umuarama,41.0,PR,Paraná,4.0,Sul,"MULTIPOLYGON (((-53.31502 -23.82851, -53.31553..."
4,4118402.0,Paranavaí,41.0,PR,Paraná,4.0,Sul,"MULTIPOLYGON (((-52.42091 -23.14106, -52.41414..."
5,4104303.0,Campo Mourão,41.0,PR,Paraná,4.0,Sul,"MULTIPOLYGON (((-52.37089 -23.98980, -52.37816..."
6,4108601.0,Goioerê,41.0,PR,Paraná,4.0,Sul,"MULTIPOLYGON (((-53.04900 -24.25452, -53.05002..."
7,4104808.0,Cascavel,41.0,PR,Paraná,4.0,Sul,"MULTIPOLYGON (((-53.33281 -25.36348, -53.32708..."
8,4105508.0,Cianorte,41.0,PR,Paraná,4.0,Sul,"MULTIPOLYGON (((-52.50163 -23.64750, -52.49811..."
9,4102000.0,Assis Chateaubriand,41.0,PR,Paraná,4.0,Sul,"MULTIPOLYGON (((-53.52941 -24.24684, -53.53387..."


Now we know our GeoDataFrame works, let us move on to the next part.

# Part II: EDA

Let's start by doing some descriptive stats and correlations between variables that might be of interest if we intend to sell b2b products to the companies.

In [6]:
print('test')

test


# Open Questions

<br>

### Any conclusions based on the number of companies created by date, month, year etc.?

Yes, blalblabla




In [7]:
#question 1 code

print('test')

test


blablabla

### Does the number of branches for a CNPJ provided on the dataset is equal to the actual number produced by the dataset?

We can check for that.


In [8]:
#question 2 code

print('test')

test


In fact, there is a big difference.

### What about the business activities (CNAE, MCC)? What are the main types? Can they be aggregated into fewer groups?

the most common are blablablala.

Yes, they can as we can see here.


In [9]:
#question 3 code

print('test')

test


### What are the differences between the cities / zip codes?

The city code is a unique identifier issued by IBGE to make it easier to join city data from different sources.

The ZIP code (in Brazil, CEP) is an area code for postal service. Some insights might be derived from it, depending on business needs.

### Is it possible to catch any spatial relationships? Which visualizations would be best in this case?

Barely. We can calculate the distance between the ZIP code centroids for any given two ZIP codes. This would roughly give us the distance between two companies, but a lot of cities (<50k inhabitants) have only one ZIP code.

For a city like Curitiba with multiple ZIP codes, if we wanted to spread our product between businesses to help word-of-mouth marketing instead of selling to many neighbouring companies, we might recommend the top businesses for each ZIP code instead of the entire city.

Besides that, we can only plot some map charts with city-level granularity, some examples below.

In [10]:
#question 5 code

print('test')

test


### If you were to make any model from the data, which one do you think makes sense?

If we have several variants of a new product and expect each one of them to have a different persona, then we might run a clustering model. Then we can try matching each cluster to a persona and be more precise with our sales.

However, let's say we have an existing product and would like to enter the Paraná state market. We might be interested in running a classification model. An example would be a logistic regression (0 = no deal and 1 = deal from CRM data) to rank the most probable leads to close a deal.

If we did so, we might not even need to define a decision boundary because the probability output is already useful in ranking leads, maybe even more so.

Even a linear model where the target is revenue from each company might help in recommending leads, so we can know which ones would be more propense to buy big. However, that only works if we have many products with different values, else it would be a waste.

In that case, we could even plot the results from the regression and the classification models to get the ones with high probability of buying the product and spending a lot.

Depending on the amount of products we sell, an actual recommendation system could also be useful.